In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import json
import os
from datetime import datetime

In [ ]:
# Load dataset
df = pd.read_csv('finance_data.csv')
print(f"Dataset loaded: {df.shape[0]} records")

In [ ]:
# Encode categorical columns
le_occ = LabelEncoder()
le_city = LabelEncoder()
df['Occupation_encoded'] = le_occ.fit_transform(df['Occupation'])
df['City_Tier_encoded'] = le_city.fit_transform(df['City_Tier'])

In [ ]:
# Create expense columns
expense_cols = ['Rent','Loan_Repayment','Insurance','Groceries','Transport','Eating_Out',
                'Entertainment','Utilities','Healthcare','Education','Miscellaneous']
df['Total_Expenses'] = df[expense_cols].sum(axis=1)

In [ ]:
# Train savings prediction model
features = [
    'Income', 'Age', 'Dependents', 'Occupation_encoded', 'City_Tier_encoded',
    'Total_Expenses', 'Desired_Savings_Percentage', 'Disposable_Income'
]
X = df[features]
y = df['Desired_Savings']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_savings = GradientBoostingRegressor(random_state=42)
model_savings.fit(X_train, y_train)
print("Savings model trained")

In [ ]:
# Train overspending risk model
df['Overspend'] = (df['Total_Expenses'] > 0.8 * df['Income']).astype(int)
X_risk = df[['Income', 'Age', 'Dependents', 'Occupation_encoded', 'City_Tier_encoded']]
y_risk = df['Overspend']
clf_risk = RandomForestClassifier(random_state=42)
clf_risk.fit(X_risk, y_risk)
print("Risk model trained")

In [ ]:
# User prediction function
_default_payload = {
    "Income": 60000.0,
    "Age": 30.0,
    "Dependents": 1.0,
    "Occupation_encoded": 1.0,
    "City_Tier_encoded": 1.0,
    "Total_Expenses": 25000.0,
    "Desired_Savings_Percentage": 15.0,
    "Disposable_Income": 35000.0,
}
_default_labels = {
    "Occupation": "Salaried",
    "City_Tier": "Tier 2",
}

def _normalize_payload(payload: dict) -> dict:
    values = _default_payload.copy()
    for key in values:
        try:
            values[key] = float(payload.get(key, values[key]))
        except (TypeError, ValueError):
            values[key] = values[key]
    return values

def run_user_prediction(payload: dict) -> dict:
    values = _normalize_payload(payload)
    occupation_label = payload.get("Occupation", _default_labels["Occupation"])
    city_label = payload.get("City_Tier", _default_labels["City_Tier"])
    
    # Encode categorical values
    if occupation_label:
        try:
            values["Occupation_encoded"] = float(le_occ.transform([occupation_label])[0])
        except Exception:
            pass
    if city_label:
        try:
            values["City_Tier_encoded"] = float(le_city.transform([city_label])[0])
        except Exception:
            pass

    # Predict savings
    feature_row = pd.DataFrame([{f: values.get(f, _default_payload[f]) for f in features}])
    predicted = float(model_savings.predict(feature_row)[0])

    # Predict overspend probability
    overspend_payload = pd.DataFrame([{
        "Income": values["Income"],
        "Age": values["Age"],
        "Dependents": values["Dependents"],
        "Occupation_encoded": values["Occupation_encoded"],
        "City_Tier_encoded": values["City_Tier_encoded"],
    }])
    overspend_prob = None
    try:
        overspend_prob = float(clf_risk.predict_proba(overspend_payload)[0][1])
    except Exception:
        overspend_prob = None

    result = {
        "predicted_desired_savings": round(predicted, 2),
        "overspend_probability": overspend_prob,
        "input": values,
        "generated_at": datetime.utcnow().isoformat() + "Z",
    }
    
    # Write result to file
    with open("user_prediction.json", "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    return result

In [ ]:
# Execute user prediction if payload provided
if os.environ.get("USER_INPUT_PAYLOAD"):
    payload = json.loads(os.environ["USER_INPUT_PAYLOAD"])
    user_result = run_user_prediction(payload)
    print("USER_RESULT", json.dumps(user_result))
else:
    print("No user input payload provided")